In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [4]:
class Vocab(object):
    
    def __init__(self, filename):
        self.idx_to_word = {}
        self.word_to_idx = {}
        self.filename = filename
        self.unk_vec = None
        self.dim = None
        with open(filename) as f:
            idx = 0;
            for line in f:
                line = line.split()
                self.idx_to_word[idx] = line[0]
                self.word_to_idx[line[0]] = idx
                if not self.dim:
                    self.dim = len(line[1:])
                idx += 1
        
        self.embedding_matrix = torch.zeros(len(self.idx_to_word)+2, dim)
        
        with open(filename) as f:
            idx = 1;
            for line in f:
                line = line.split()
                self.embedding_matrix[idx] = torch.tensor(list(map(float, line[1:])))
                idx += 1
            self.unk_vec = torch.sum(self.embedding_matrix, 0)/(len(self.idx_to_word))
            self.embedding_matrix[len(self.idx_to_word)+1] = self.unk_vec
        
    def embedding(self, input_seq):
        MAX_LEN = input_seq.size()[0]
        batch_size = input_seq.size()[1]
        embedded = torch.zeros(MAX_LEN, batch_size, self.dim)
        for i in range(MAX_LEN):
            for j in range(batch_size):
                embedded[i,j,:] = embedding_matrix[input_seq[i, j]]
        return embedded
    
    def encode(self, sentence):
        encoded = torch.zeros(len(sentence))
        idx=0
        for word in sentence:
            if word in word_to_idx:
                encoded[idx] = word_to_idx[word]
            else:
                encoded[idx] = len(word_to_idx)+1
            idx += 1
        
        return encoded
    
    def decode(self, sentence):
        decoded = torch.zeros(*sentence.size())
        idx = 0
        for word in sentence:
            decoded[idx] = idx_to_word[word]
        
        return decoded
    

In [ ]:
def batcher(list_sentence, MAX_LEN, batch_size, test_ratio=0.2):
    total = len(list_sentence)
    train_len = total*(1-test_ratio)
    test_len = total - train_length
    train_sentence = list_sentence[:train_len]
    test_sentence = list_sentence[train_len:]
    train = []
    test = []
    idx = 0;
    while True:
        pos = idx
        if pos == train_len:
            break;
        if (idx + batch_size <= train_len):
            next_pos = idx+batch_size
        else:
            next_pos = train_len
        t_batch = torch.zeros(MAX_LEN, next_pos - pos)
        tmp_batch = train_sentence[pos:next_pos]
        length = []
        mask = torch.zeros(MAX_LEN, next_pos-pos)
        for batch_n in range(next_pos - pos):
            l = MAX_LEN if len(tmp_batch[batch_n]) > MAX_LEN else len(tmp_batch[batch_n])
            length.append(l)
            t_batch[:l, batch_n] = tmp_batch[batch_n][:l]
            mask[:l, batch_n] = torch.ones(l)
        train.append((t_batch, mask, length))
        idx = next_pos
    